In [1]:
import numpy as np
import pandas as pd
import heapq
import math
import time
from math import radians, sin, cos, sqrt, atan2

In [2]:
df = pd.read_csv('Dataset.csv')
df.head(10)

,Airline,SourceAirport,DestinationAirport,SourceAirport_City,SourceAirport_Country,SourceAirport_Latitude,SourceAirport_Longitude,SourceAirport_Altitude,DestinationAirport_City,DestinationAirport_Country,DestinationAirport_Latitude,DestinationAirport_Longitude,DestinationAirport_Altitude,Distance,FlyTime,Price
0,Aerocondor,Sochi International Airport,Kazan International Airport,Sochi,Russia,43.449902,39.956600,89,Kazan,Russia,55.606201,49.278702,411,1506.825604,2.330543,172.662342
1,Aerocondor,Astrakhan Airport,Kazan International Airport,Astrakhan,Russia,46.283298,48.006302,-65,Kazan,Russia,55.606201,49.278702,411,1040.438320,1.798405,132.955783
2,Aerocondor,Chelyabinsk Balandino Airport,Kazan International Airport,Chelyabinsk,Russia,55.305801,61.503300,769,Kazan,Russia,55.606201,49.278702,411,770.508500,1.906214,78.543730
3,Aerocondor,Domodedovo International Airport,Kazan International Airport,Moscow,Russia,55.408798,37.906300,588,Kazan,Russia,55.606201,49.278702,411,715.649350,1.699983,170.633416
4,Aerocondor,Belgorod International Airport,Kazan International Airport,Belgorod,Russia,50.643799,36.590099,735,Kazan,Russia,55.606201,49.278702,411,1008.253110,1.573957,183.681933
5,Aerocondor,Pulkovo Airport,Kazan International Airport,St. Petersburg,Russia,59.800301,30.262501,78,Kazan,Russia,55.606201,49.278702,411,1216.844567,2.264405,165.368643
6,Aerocondor,Koltsovo Airport,Kazan International Airport,Yekaterinburg,Russia,56.743099,60.802700,764,Kazan,Russia,55.606201,49.278702,411,723.523204,0.943500,90.533103
7,Uzbekistan Airways,Fergana International Airport,Kazan International Airport,Fergana,Uzbekistan,40.358799,71.745003,1980,Kazan,Russia,55.606201,49.278702,411,2360.729777,3.871370,290.389052
8,Uzbekistan Airways,Samarkand Airport,Kazan International Airport,Samarkand,Uzbekistan,39.700500,66.983803,2224,Kazan,Russia,55.606201,49.278702,411,2196.998819,3.389498,258.369110
9,Uzbekistan Airways,Tashkent International Airport,Kazan International Airport,Tashkent,Uzbekistan,41.257900,69.281197,1417,Kazan,Russia,55.606201,49.278702,411,2157.778320,2.979653,284.266288


In [3]:
airport_source = 'Imam Khomeini International Airport'
airport_destination = 'Raleigh Durham International Airport'

In [4]:
class Node:
    def __init__(self, name, city, country, latitude, longitude, altitude):
        self.name = name
        self.city = city
        self.country = country
        self.latitude = latitude
        self.longitude = longitude
        self.altitude = altitude
        self.neighbors = {}

    def add_neighbor(self, neighbor, airline, distance, price, fly_time):
        self.neighbors[neighbor] = {'Airline': airline, 'Distance': distance, 'Price': price, 'FlyTime': fly_time}

In [5]:
nodes = {}
for _,row in df.iterrows():
    source = row['SourceAirport']
    destination = row['DestinationAirport']

    if source not in nodes:
        nodes[source] = Node(row['SourceAirport'],
                             row['SourceAirport_City'], row['SourceAirport_Country'],
                             row['SourceAirport_Latitude'], row['SourceAirport_Longitude'],
                             row['SourceAirport_Altitude'])
    if destination not in nodes:
        nodes[destination] = Node(row['DestinationAirport'],
                                  row['DestinationAirport_City'], row['DestinationAirport_Country'],
                                  row['DestinationAirport_Latitude'], row['DestinationAirport_Longitude'],
                                  row['DestinationAirport_Altitude'])

    nodes[source].add_neighbor(nodes[destination], row['Airline'], row['Distance'], row['Price'], row['FlyTime'])

In [6]:
def compute_weight(distance, price, flyTime):
    w_distance = 1.7
    w_price = 1.2
    w_flyTime = 1.1

    weight = (w_distance * distance) + (w_price * price) + (w_flyTime * flyTime)
    return weight

In [7]:
def dijkstra(source_name, destination_name):
    source = nodes[source_name]
    destination = nodes[destination_name]

    if source.name == destination.name:
        return {source: None}

    pervious = {n: None for n in nodes.values()}
    distances = {n: float('inf') for n in nodes.values()}

    distances[source] = 0
    priority_queue = [(0, source)]

    counter_dijkstra = 0
    while priority_queue:
        counter_dijkstra += 1
        cur_distance, cur_node = heapq.heappop(priority_queue)

        if cur_node.name == destination.name:
            return cur_distance, pervious

        for neighbor, edge in cur_node.neighbors.items():
            weight = compute_weight(edge['Distance'], edge['Price'], edge['FlyTime'])
            new_distance = cur_distance + weight
            if new_distance < distances[neighbor]:
                distances[neighbor] = new_distance
                pervious[neighbor] = cur_node
                heapq.heappush(priority_queue, (new_distance, neighbor))

In [8]:
def get_path(pervious, destination):
    path = []
    current = nodes[destination]
    while current is not None:
        path.insert(0, current)
        current = pervious[current]
    return path

In [9]:
def print_info(path):
    total_distance = 0
    total_price = 0
    total_fly_time = 0
    result_str = ""

    for i in range(len(path)):
        node = path[i]
        if i < len(path) - 1:
            next_node = path[i + 1]
            edge = node.neighbors[next_node]
            result_str += ("Flight #"+str(i + 1)+"("+node.neighbors[next_node]['Airline']+")"+"\n")
            result_str += ("From: "+node.name+" - "+node.city+", "+ node.country+"\n")
            result_str += ("To: "+next_node.name+" - "+next_node.city+", "+next_node.country+"\n")
            result_str += ("Duration: {:.2f}km".format(edge['Distance'])+"\n")
            result_str += ("Time: {:.2f}h".format(edge['FlyTime'])+"\n")
            result_str += ("Price: {:.2f}$".format(edge['Price'])+"\n")
            result_str += ("----------------------------"+"\n")

            total_distance += edge['Distance']
            total_price += edge['Price']
            total_fly_time += edge['FlyTime']

    result_str += ("Total Price: {:.2f}$".format(total_price)+"\n")
    result_str += ("Total Duration: {:.2f}km".format(total_distance)+"\n")
    result_str += ("Total Time: {:.2f}h".format(total_fly_time)+"\n")
    return result_str

In [10]:
start_time = time.time()
num_visited_nodes, pervious = dijkstra(airport_source, airport_destination)
elapsed_time = time.time()  - start_time

str_path_dijkstra = ("Dijkstra Algorithm"+"\n")
minutes, seconds = divmod(elapsed_time, 60)
str_path_dijkstra += ("Execution Time: {:.0f}m{:.5f}s".format(minutes, seconds)+"\n")
str_path_dijkstra += (".-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-"+"\n")

path = get_path(pervious, airport_destination)
str_path_dijkstra += print_info(path)
print(str_path_dijkstra)
print("number of visited nodes: ", num_visited_nodes, "\n")

Dijkstra Algorithm
Execution Time: 0m0.19917s
.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-
Flight #1(Iran Aseman Airlines)
From: Imam Khomeini International Airport - Tehran, Iran
To: Zvartnots International Airport - Yerevan, Armenia
Duration: 792.85km
Time: 1.76h
Price: 116.46$
----------------------------
Flight #2(Czech Airlines)
From: Zvartnots International Airport - Yerevan, Armenia
To: Václav Havel Airport Prague - Prague, Czech Republic
Duration: 2587.13km
Time: 4.19h
Price: 262.12$
----------------------------
Flight #3(Jet2.com)
From: Václav Havel Airport Prague - Prague, Czech Republic
To: Newcastle Airport - Newcastle, United Kingdom
Duration: 1206.01km
Time: 2.14h
Price: 171.71$
----------------------------
Flight #4(Jetstar Airways)
From: Newcastle Airport - Newcastle, United Kingdom
To: Melbourne International Airport - Melbourne, Australia
Duration: 834.64km
Time: 1.42h
Price: 117.19$
----------------------------
Flight #5(American Airlines)
From: Melbourne Interna

In [11]:
def calculate_distance(lat1, lon1, alt1, lat2, lon2, alt2):
    # Convert coordinates from degrees to radians
    lat1_rad = radians(lat1)
    lon1_rad = radians(lon1)
    lat2_rad = radians(lat2)
    lon2_rad = radians(lon2)

    # Earth radius in kilometers
    earth_radius = 6371.0

    # Calculate differences in coordinates
    dlat = lat2_rad - lat1_rad
    dlon = lon2_rad - lon1_rad
    dalt = alt2 - alt1

    # Haversine formula
    a = sin(dlat/2)**2 + cos(lat1_rad) * cos(lat2_rad) * sin(dlon/2)**2
    c = 2 * atan2(sqrt(a), sqrt(1-a))

    # Calculate distance
    distance = earth_radius * c

    # Add altitude difference
    distance += abs(dalt)
    return distance

In [12]:
def heurestic_distance(source, destination):
    d1_source = source.latitude
    d2_source = source.longitude
    d3_source = source.altitude

    d1_destination = destination.latitude
    d2_destination = destination.longitude
    d3_destination = destination.altitude

    heuristic = math.sqrt(((d1_source - d1_destination) ** 2) + ((d2_source - d2_destination) ** 2) +
                          ((d3_source - d3_destination) ** 2))
    #heuristic = calculate_distance(d1_source, d2_source, d3_source, d1_destination, d2_destination, d3_destination)
    return heuristic

In [13]:
def a_star(source_name, destination_name):

    source = nodes[source_name]
    destination = nodes[destination_name]

    g = {n: float('inf') for n in nodes.values()}
    f = {n: float('inf') for n in nodes.values()}
    pervious = {n: None for n in nodes.values()}
    priority_queue = [(0, source)]

    g[source] = 0
    f[source] = heurestic_distance(source, destination)

    counter_a_star = 0
    while priority_queue:
        counter_a_star += 1
        cur_distance, cur_node = heapq.heappop(priority_queue)

        if cur_node.name == destination.name:
            return counter_a_star, pervious

        for neighbor, edge in cur_node.neighbors.items():
            weight = edge['Distance']
            new_distance = g[cur_node] + weight
            if new_distance < g[neighbor]:
                g[neighbor] = new_distance
                f[neighbor] = g[neighbor] + heurestic_distance(neighbor, destination)
                pervious[neighbor] = cur_node
                heapq.heappush(priority_queue, (f[neighbor], neighbor))

In [14]:
start_time = time.time()
num_visited_nodes, pervious = a_star(airport_source, airport_destination)
elapsed_time = time.time()  - start_time

str_path_a_star = ("A* Algorithm"+"\n")
minutes, seconds = divmod(elapsed_time, 60)
str_path_a_star += ("Execution Time: {:.0f}m{:.5f}s".format(minutes, seconds)+"\n")
str_path_a_star += (".-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-"+"\n")

path = get_path(pervious, airport_destination)
str_path_a_star += print_info(path)
print(str_path_a_star)
print("number of visited nodes: ", num_visited_nodes)

A* Algorithm
Execution Time: 0m0.09887s
.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-
Flight #1(Iran Aseman Airlines)
From: Imam Khomeini International Airport - Tehran, Iran
To: Zvartnots International Airport - Yerevan, Armenia
Duration: 792.85km
Time: 1.76h
Price: 116.46$
----------------------------
Flight #2(Czech Airlines)
From: Zvartnots International Airport - Yerevan, Armenia
To: Václav Havel Airport Prague - Prague, Czech Republic
Duration: 2587.13km
Time: 4.19h
Price: 262.12$
----------------------------
Flight #3(Jet2.com)
From: Václav Havel Airport Prague - Prague, Czech Republic
To: Newcastle Airport - Newcastle, United Kingdom
Duration: 1206.01km
Time: 2.14h
Price: 171.71$
----------------------------
Flight #4(Jetstar Airways)
From: Newcastle Airport - Newcastle, United Kingdom
To: Melbourne International Airport - Melbourne, Australia
Duration: 834.64km
Time: 1.42h
Price: 117.19$
----------------------------
Flight #5(American Airlines)
From: Melbourne International